# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0     hamilton  39.1834  -84.5333     25.87        81           0   
1        1     ust-nera  64.5667  143.2000     18.89        54         100   
2        2  al kharijah  25.4514   30.5464     37.41        10          26   
3        3         tefe  -3.3542  -64.7114     22.06        98          93   
4        4        iwaki  37.0500  140.8833     28.76        86         100   

   Wind Speed Country        Date  
0        5.14      US  1719997661  
1        1.36      RU  1719997927  
2        1.66      EG  1719997928  
3        1.00      BR  1719997929  
4        2.00      JP  1719997252

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo =True, 
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7)

# Display the map
display(map_plot)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df["Max Temp"]<27)& (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df

City_ID                  City      Lat       Lng  Max Temp  Humidity  \
19        19                newman  37.3138 -121.0208     26.27        66   
24        24             jamestown  42.0970  -79.2353     21.77        67   
136      136               tsiombe -25.3000   45.4833     25.62        44   
259      259                darwin -12.4611  130.8418     26.99        60   
265      265               fernley  39.6080 -119.2518     22.22        16   
322      322             morondava -20.2833   44.2833     24.19        49   
361      361                tiznit  29.5833   -9.5000     24.20        47   
365      365      kill devil hills  36.0307  -75.6760     22.77        89   
384      384             ambovombe -25.1667   46.0833     24.17        52   
416      416           basse-terre  16.1667  -61.6667     26.58        74   
426      426  warm mineral springs  27.0598  -82.2601     26.02        98   
427      427               trumann  35.6737  -90.5073     24.05        58   
428      428               divrigi  39.3710   38.1137     26.52        27   
528      528          stephenville  32.2207  -98.2023     26.21        72   
564      564                laguna  38.4210 -121.4238     26.99        50   

     Cloudiness  Wind Speed Country        Date  
19            0        1.37      US  1719997952  
24            0        3.60      US  1719997959  
136           0        4.33      MG  1719998123  
259           0        2.57      AU  1719998106  
265           0        1.79      US  1719998336  
322           0        3.05      MG  1719998425  
361           0        2.14      MA  1719998484  
365           0        0.00      US  1719998489  
384           0        3.93      MG  1719998516  
416           0        4.12      GP  1719998559  
426           0        3.09      US  1719998572  
427           0        3.09      US  1719998574  
428           0        1.06      TR  1719998575  
528           0        2.06      US  1719998716  
564           0        1.54      US  1719998523

### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
19                 newman      US  37.3138 -121.0208        66           
24              jamestown      US  42.0970  -79.2353        67           
136               tsiombe      MG -25.3000   45.4833        44           
259                darwin      AU -12.4611  130.8418        60           
265               fernley      US  39.6080 -119.2518        16           
322             morondava      MG -20.2833   44.2833        49           
361                tiznit      MA  29.5833   -9.5000        47           
365      kill devil hills      US  36.0307  -75.6760        89           
384             ambovombe      MG -25.1667   46.0833        52           
416           basse-terre      GP  16.1667  -61.6667        74           
426  warm mineral springs      US  27.0598  -82.2601        98           
427               trumann      US  35.6737  -90.5073        58           
428               divrigi      TR  39.3710   38.1137        27           
528          stephenville      US  32.2207  -98.2023        72           
564                laguna      US  38.4210 -121.4238        50

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "filter": "",
    "limit": limit,
    "bias": "",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
tsiombe - nearest hotel: No hotel found
darwin - nearest hotel: Mantra Pantanas Darwin
fernley - nearest hotel: Best Western Fernley Inn
morondava - nearest hotel: Hotel Sharon Menabe
tiznit - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
ambovombe - nearest hotel: No hotel found
basse-terre - nearest hotel: Le Créoliles
warm mineral springs - nearest hotel: No hotel found
trumann - nearest hotel: No hotel found
divrigi - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites


City Country      Lat       Lng  Humidity  \
19                 newman      US  37.3138 -121.0208        66   
24              jamestown      US  42.0970  -79.2353        67   
136               tsiombe      MG -25.3000   45.4833        44   
259                darwin      AU -12.4611  130.8418        60   
265               fernley      US  39.6080 -119.2518        16   
322             morondava      MG -20.2833   44.2833        49   
361                tiznit      MA  29.5833   -9.5000        47   
365      kill devil hills      US  36.0307  -75.6760        89   
384             ambovombe      MG -25.1667   46.0833        52   
416           basse-terre      GP  16.1667  -61.6667        74   
426  warm mineral springs      US  27.0598  -82.2601        98   
427               trumann      US  35.6737  -90.5073        58   
428               divrigi      TR  39.3710   38.1137        27   
528          stephenville      US  32.2207  -98.2023        72   
564                laguna      US  38.4210 -121.4238        50   

                       Hotel Name  
19                 No hotel found  
24           DoubleTree Jamestown  
136                No hotel found  
259        Mantra Pantanas Darwin  
265      Best Western Fernley Inn  
322           Hotel Sharon Menabe  
361                No hotel found  
365     Mariner Days Inn & Suites  
384                No hotel found  
416                  Le Créoliles  
426                No hotel found  
427                No hotel found  
428                No hotel found  
528                No hotel found  
564  Holiday Inn Express & Suites

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo =True, 
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    hover_cols='all'
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)